In [1]:
from datasets import Dataset, DatasetDict
from tqdm import tqdm
import pandas as pd
import json
import wandb
import os

/home/kyobrien/conda/envs/eval-aug/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
project_name = "healthyml/In-Context Domain Transfer Improves Out-of-Domain Robustness"
table_paths = {
    # BOSS Sentiment
    # ID
    # BERT
    "BOSS_Sentiment_ID_BERT_Insert": "run-vj8tss18-validation_aug_insert_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_ID_BERT_Substitute": "",
    "BOSS_Sentiment_ID_BERT_Translate": "",
    "BOSS_Sentiment_ID_BERT_Paraphrase": "",
    "BOSS_Sentiment_ID_BERT_ICR": "run-yanx1dus-validation_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",

    # T5
    "BOSS_Sentiment_ID_T5_Insert": "run-vj8tss18-validation_aug_insert_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_ID_T5_Substitute": "run-vj8tss18-validation_aug_substitute_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_ID_T5_Translate": "run-vj8tss18-validation_aug_backtranslate_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_ID_T5_Paraphrase": "",
    "BOSS_Sentiment_ID_T5_ICR": "",

    # Falcon
    "BOSS_Sentiment_ID_Falcon_Insert": "run-j11olfw3-validation_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_ID_Falcon_Substitute": "run-j11olfw3-validation_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_ID_Falcon_Translate": "run-j11olfw3-validation_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_ID_Falcon_Paraphrase": "",
    "BOSS_Sentiment_ID_Falcon_ICR": "",

    # SST5
    # BERT
    "BOSS_Sentiment_SST5_BERT_Insert": "run-hhtbcw8x-sst5_aug_insert_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SST5_BERT_Substitute": "run-hhtbcw8x-sst5_aug_substitute_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SST5_BERT_Translate": "run-hhtbcw8x-sst5_aug_backtranslate_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SST5_BERT_Paraphrase": "run-hhtbcw8x-sst5_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SST5_BERT_ICR": "run-hhtbcw8x-sst5_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",

    # T5
    "BOSS_Sentiment_SST5_T5_Insert": "run-hhtbcw8x-sst5_aug_insert_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SST5_T5_Substitute": "run-hhtbcw8x-sst5_aug_substitute_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SST5_T5_Translate": "run-hhtbcw8x-sst5_aug_backtranslate_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SST5_T5_Paraphrase": "run-hhtbcw8x-sst5_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SST5_T5_ICR": "run-hhtbcw8x-sst5_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentt5large_style_logs:v0",

    # Falcon
    "BOSS_Sentiment_SST5_Falcon_Insert": "run-hhtbcw8x-sst5_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_SST5_Falcon_Substitute": "run-hhtbcw8x-sst5_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_SST5_Falcon_Translate": "run-hhtbcw8x-sst5_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_SST5_Falcon_Paraphrase": "run-hhtbcw8x-sst5_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_SST5_Falcon_ICR": "run-hhtbcw8x-sst5_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",

    # SemEval
    # BERT
    "BOSS_Sentiment_SemEval_BERT_Insert": "run-hhtbcw8x-semval_aug_insert_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SemEval_BERT_Substitute": "run-hhtbcw8x-semval_aug_substitute_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SemEval_BERT_Translate": "run-hhtbcw8x-semval_aug_backtranslate_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SemEval_BERT_Paraphrase": "run-hhtbcw8x-semval_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SemEval_BERT_ICR": "run-hhtbcw8x-semval_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",

    # T5
    "BOSS_Sentiment_SemEval_T5_Insert": "run-hhtbcw8x-semval_aug_insert_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SemEval_T5_Substitute": "run-hhtbcw8x-semval_aug_substitute_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SemEval_T5_Translate": "run-hhtbcw8x-semval_aug_backtranslate_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SemEval_T5_Paraphrase": "run-hhtbcw8x-semval_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SemEval_T5_ICR": "run-hhtbcw8x-semval_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentt5large_style_logs:v0",

    # Falcon
    "BOSS_Sentiment_SemEval_Falcon_Insert": "run-hhtbcw8x-semval_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_SemEval_Falcon_Substitute": "run-hhtbcw8x-semval_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_SemEval_Falcon_Translate": "run-hhtbcw8x-semval_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_SemEval_Falcon_Paraphrase": "run-hhtbcw8x-semval_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_SemEval_Falcon_ICR": "run-hhtbcw8x-semval_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",

    # Dynasent
    # BERT
    "BOSS_Sentiment_Dynasent_BERT_Insert": "run-hhtbcw8x-dynasent_aug_insert_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Dynasent_BERT_Substitute": "run-hhtbcw8x-dynasent_aug_substitute_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Dynasent_BERT_Translate": "run-hhtbcw8x-dynasent_aug_backtranslate_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Dynasent_BERT_Paraphrase": "run-hhtbcw8x-dynasent_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Dynasent_BERT_ICR": "run-hhtbcw8x-dynasent_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",

    # T5
    "BOSS_Sentiment_Dynasent_T5_Insert": "run-hhtbcw8x-dynasent_aug_insert_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_Dynasent_T5_Substitute": "run-hhtbcw8x-dynasent_aug_substitute_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_Dynasent_T5_Translate": "run-hhtbcw8x-dynasent_aug_backtranslate_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_Dynasent_T5_Paraphrase": "run-hhtbcw8x-dynasent_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_Dynasent_T5_ICR": "run-hhtbcw8x-dynasent_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentt5large_style_logs:v0",

    # Falcon
    "BOSS_Sentiment_Dynasent_Falcon_Insert": "run-hhtbcw8x-dynasent_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_Dynasent_Falcon_Substitute": "run-hhtbcw8x-dynasent_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_Dynasent_Falcon_Translate": "run-hhtbcw8x-dynasent_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_Dynasent_Falcon_Paraphrase": "run-hhtbcw8x-dynasent_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_Dynasent_Falcon_ICR": "run-hhtbcw8x-dynasent_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",

    # BOSS Toxicity
    # ID
    # BERT
    "BOSS_Toxicity_ID_BERT_Insert": "",
    "BOSS_Toxicity_ID_BERT_Substitute": "",
    "BOSS_Toxicity_ID_BERT_Translate": "",
    "BOSS_Toxicity_ID_BERT_Paraphrase": "",
    "BOSS_Toxicity_ID_BERT_ICR": "run-dvdcuuij-validation_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",

    # T5
    "BOSS_Toxicity_ID_T5_Insert": "",
    "BOSS_Toxicity_ID_T5_Substitute": "",
    "BOSS_Toxicity_ID_T5_Translate": "",
    "BOSS_Toxicity_ID_T5_Paraphrase": "",
    "BOSS_Toxicity_ID_T5_ICR": "",

    # Falcon
    "BOSS_Toxicity_ID_Falcon_Insert": "run-bo8gbtun-validation_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_ID_Falcon_Substitute": "",
    "BOSS_Toxicity_ID_Falcon_Translate": "",
    "BOSS_Toxicity_ID_Falcon_Paraphrase": "",
    "BOSS_Toxicity_ID_Falcon_ICR": "",

    # Toxigen
    # BERT
    "BOSS_Toxicity_Toxigen_BERT_Insert": "run-lixlm6qh-toxigen_aug_insert_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Toxigen_BERT_Substitute": "run-lixlm6qh-toxigen_aug_substitute_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Toxigen_BERT_Translate": "run-lixlm6qh-toxigen_aug_backtranslate_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Toxigen_BERT_Paraphrase": "run-lixlm6qh-toxigen_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Toxigen_BERT_ICR": "run-lixlm6qh-toxigen_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",

    # T5
    "BOSS_Toxicity_Toxigen_T5_Insert": "run-lixlm6qh-toxigen_aug_insert_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_Toxigen_T5_Substitute": "run-lixlm6qh-toxigen_aug_substitute_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_Toxigen_T5_Translate": "run-lixlm6qh-toxigen_aug_backtranslate_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_Toxigen_T5_Paraphrase": "run-lixlm6qh-toxigen_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_Toxigen_T5_ICR": "run-lixlm6qh-toxigen_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",

    # Falcon
    "BOSS_Toxicity_Toxigen_Falcon_Insert": "run-lixlm6qh-toxigen_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_Toxigen_Falcon_Substitute": "run-lixlm6qh-toxigen_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_Toxigen_Falcon_Translate": "run-lixlm6qh-toxigen_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_Toxigen_Falcon_Paraphrase": "run-lixlm6qh-toxigen_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_Toxigen_Falcon_ICR": "run-lixlm6qh-toxigen_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",

    # ImplicitHate
    # BERT
    "BOSS_Toxicity_ImplicitHate_BERT_Insert": "run-lixlm6qh-implicit_hate_aug_insert_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_BERT_Substitute": "run-lixlm6qh-implicit_hate_aug_substitute_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_BERT_Translate": "run-lixlm6qh-implicit_hate_aug_backtranslate_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_BERT_Paraphrase": "run-lixlm6qh-implicit_hate_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_BERT_ICR": "run-lixlm6qh-implicit_hate_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",

    # T5
    "BOSS_Toxicity_ImplicitHate_T5_Insert": "run-lixlm6qh-implicit_hate_aug_insert_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_T5_Substitute": "run-lixlm6qh-implicit_hate_aug_substitute_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_T5_Translate": "run-lixlm6qh-implicit_hate_aug_backtranslate_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_T5_Paraphrase": "run-lixlm6qh-implicit_hate_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_T5_ICR": "run-lixlm6qh-implicit_hate_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",

    # Falcon
    "BOSS_Toxicity_ImplicitHate_Falcon_Insert": "run-lixlm6qh-implicit_hate_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_Falcon_Substitute": "run-lixlm6qh-implicit_hate_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_Falcon_Translate": "run-lixlm6qh-implicit_hate_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_Falcon_Paraphrase": "",
    "BOSS_Toxicity_ImplicitHate_Falcon_ICR": "",

    # AdvCivil
    # BERT
    "BOSS_Toxicity_AdvCivil_BERT_Insert": "run-lixlm6qh-adv_civil_aug_insert_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_BERT_Substitute": "run-lixlm6qh-adv_civil_aug_substitute_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_BERT_Translate": "run-lixlm6qh-adv_civil_aug_backtranslate_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_BERT_Paraphrase": "run-lixlm6qh-adv_civil_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_BERT_ICR": "run-lixlm6qh-adv_civil_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",

    # T5
    "BOSS_Toxicity_AdvCivil_T5_Insert": "run-lixlm6qh-adv_civil_aug_insert_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_T5_Substitute": "run-lixlm6qh-adv_civil_aug_substitute_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_T5_Translate": "run-lixlm6qh-adv_civil_aug_backtranslate_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_T5_Paraphrase": "run-lixlm6qh-adv_civil_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_T5_ICR": "run-lixlm6qh-adv_civil_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",

    # Falcon
    "BOSS_Toxicity_AdvCivil_Falcon_Insert": "run-lixlm6qh-adv_civil_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_Falcon_Substitute": "run-lixlm6qh-adv_civil_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_Falcon_Translate": "run-lixlm6qh-adv_civil_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_Falcon_Paraphrase": "run-lixlm6qh-adv_civil_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_Falcon_ICR": "run-lixlm6qh-adv_civil_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",

    # AG News Tweets
    # ID
    # BERT
    "AgNewsTweets_ID_BERT_Insert": "run-i2vkae2s-validation_aug_insert_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_ID_BERT_Substitute": "run-i2vkae2s-validation_aug_substitute_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_ID_BERT_Translate": "run-i2vkae2s-validation_aug_backtranslate_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_ID_BERT_Paraphrase": "run-i2vkae2s-validation_stabilityaiStableBeluga7b_random_0_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_ID_BERT_ICR": "run-i2vkae2s-validation_stabilityaiStableBeluga7b_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",

    # T5
    "AgNewsTweets_ID_T5_Insert": "run-i2vkae2s-validation_aug_insert_random_16_Kyle1668agnewst5large_style_logs:v0",
    "AgNewsTweets_ID_T5_Substitute": "run-i2vkae2s-validation_aug_substitute_random_16_Kyle1668agnewst5large_style_logs:v0",
    "AgNewsTweets_ID_T5_Translate": "run-i2vkae2s-validation_aug_backtranslate_random_16_Kyle1668agnewst5large_style_logs:v0",
    "AgNewsTweets_ID_T5_Paraphrase": "run-i2vkae2s-validation_stabilityaiStableBeluga7b_random_0_Kyle1668agnewst5large_style_logs:v0",
    "AgNewsTweets_ID_T5_ICR": "run-i2vkae2s-validation_stabilityaiStableBeluga7b_random_16_Kyle1668agnewst5large_style_logs:v0",

    # Falcon
    "AgNewsTweets_ID_Falcon_Insert": "run-i2vkae2s-validation_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "AgNewsTweets_ID_Falcon_Substitute": "run-i2vkae2s-validation_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "AgNewsTweets_ID_Falcon_Translate": "run-i2vkae2s-validation_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "AgNewsTweets_ID_Falcon_Paraphrase": "run-i2vkae2s-validation_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
    "AgNewsTweets_ID_Falcon_ICR": "run-i2vkae2s-validation_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",

    # Tweets
    # BERT
    "AgNewsTweets_Tweets_BERT_Insert": "run-xdipkawo-test_aug_insert_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_Tweets_BERT_Substitute": "run-xdipkawo-test_aug_substitute_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_Tweets_BERT_Translate": "run-xdipkawo-test_aug_backtranslate_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_Tweets_BERT_Paraphrase": "run-xdipkawo-test_stabilityaiStableBeluga7b_random_0_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_Tweets_BERT_ICR": "run-xdipkawo-test_stabilityaiStableBeluga7b_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",

    # T5
    "AgNewsTweets_Tweets_T5_Insert": "run-xdipkawo-test_aug_insert_random_16_Kyle1668agnewst5large_style_logs:v0",
    "AgNewsTweets_Tweets_T5_Substitute": "run-xdipkawo-test_aug_substitute_random_16_Kyle1668agnewst5large_style_logs:v0",
    "AgNewsTweets_Tweets_T5_Translate": "run-xdipkawo-test_aug_backtranslate_random_16_Kyle1668agnewst5large_style_logs:v0",
    "AgNewsTweets_Tweets_T5_Paraphrase": "run-xdipkawo-test_stabilityaiStableBeluga7b_random_0_Kyle1668agnewst5large_style_logs:v0",
    "AgNewsTweets_Tweets_T5_ICR": "run-xdipkawo-test_stabilityaiStableBeluga7b_random_16_Kyle1668agnewst5large_style_logs:v0",

    # Falcon
    "AgNewsTweets_Tweets_Falcon_Insert": "run-xdipkawo-test_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "AgNewsTweets_Tweets_Falcon_Substitute": "run-xdipkawo-test_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "AgNewsTweets_Tweets_Falcon_Translate": "run-xdipkawo-test_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "AgNewsTweets_Tweets_Falcon_Paraphrase": "run-xdipkawo-test_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
    "AgNewsTweets_Tweets_Falcon_ICR": "",
}

print(json.dumps(table_paths, indent=4))

{
    "BOSS_Sentiment_ID_BERT_Insert": "run-vj8tss18-validation_aug_insert_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_ID_BERT_Substitute": "",
    "BOSS_Sentiment_ID_BERT_Translate": "",
    "BOSS_Sentiment_ID_BERT_Paraphrase": "",
    "BOSS_Sentiment_ID_BERT_ICR": "run-yanx1dus-validation_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_ID_T5_Insert": "run-vj8tss18-validation_aug_insert_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_ID_T5_Substitute": "run-vj8tss18-validation_aug_substitute_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_ID_T5_Translate": "run-vj8tss18-validation_aug_backtranslate_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_ID_T5_Paraphrase": "",
    "BOSS_Sentiment_ID_T5_ICR": "",
    "BOSS_Sentiment_ID_Falcon_Insert": "run-j11olfw3-validation_aug_insert_random_16_tiiuaefalcon7binstru

In [3]:
run = wandb.init()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kyledevinobrien1 (healthyml). Use `wandb login --relogin` to force relogin


In [4]:
def get_local_logs_path(directory):
    for contents in os.listdir(directory):
        if contents.endswith("style_logs.table.json"):
            return os.path.join(directory, contents)

        if os.path.isdir(os.path.join(directory, contents)):
            return get_local_logs_path(os.path.join(directory, contents))

print(get_local_logs_path("data/BOSS_Sentiment_Dynasent_BERT_Insert"))

data/BOSS_Sentiment_Dynasent_BERT_Insert/dynasent_aug_insert_random_16_Kyle1668/boss-sentiment-bert-base-uncased_style_logs.table.json


In [5]:
# for each table path, run.use_artifact and download the artifact.
local_paths = {}
missing_tables = []
for table_name, table_path in tqdm(list(table_paths.items())[:]):
    if table_path == "":
        missing_tables.append(table_name)
        continue

    download_path = f"{project_name}/{table_path}"
    print(f"Downloading {table_name} --- {download_path}")
    artifact = run.use_artifact(download_path, type="run_table")
    artifact_dir = artifact.download(root=f"data/{table_name}")
    file_name = os.listdir(artifact_dir)[0]
    local_paths[table_name] = get_local_logs_path(f"data/{table_name}")

print(f"\nMissing tables: {missing_tables}")
print(json.dumps(local_paths, indent=4))

  0%|          | 0/150 [00:00<?, ?it/s]

wandb: Downloading large artifact run-vj8tss18-validation_aug_insert_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0, 997.68MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:2.0
  1%|          | 1/150 [00:02<06:59,  2.82s/it]

wandb: Downloading large artifact run-yanx1dus-validation_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0, 180.63MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.5
  3%|▎         | 5/150 [00:04<01:40,  1.44it/s]

wandb: Downloading large artifact run-vj8tss18-validation_aug_insert_random_16_Kyle1668bosssentimentt5large_style_logs:v0, 828.11MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:1.6
  4%|▍         | 6/150 [00:06<02:39,  1.11s/it]

wandb: Downloading large artifact run-vj8tss18-validation_aug_substitute_random_16_Kyle1668bosssentimentt5large_style_logs:v0, 817.40MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:1.6
  5%|▍         | 7/150 [00:08<03:24,  1.43s/it]

...

wandb: Downloading large artifact run-xdipkawo-test_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0, 228.25MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:2.9
 97%|█████████▋| 146/150 [04:59<00:15,  3.85s/it]

wandb: Downloading large artifact run-xdipkawo-test_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0, 224.47MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:3.0
 98%|█████████▊| 147/150 [05:02<00:11,  3.81s/it]

wandb: Downloading large artifact run-xdipkawo-test_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0, 224.00MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:2.6
 99%|█████████▊| 148/150 [05:06<00:07,  3.69s/it]

wandb: Downloading large artifact run-xdipkawo-test_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0, 227.67MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:2.9
100%|██████████| 150/150 [05:09<00:00,  2.07s/it]


Missing tables: ['BOSS_Sentiment_ID_BERT_Substitute', 'BOSS_Sentiment_ID_BERT_Translate', 'BOSS_Sentiment_ID_BERT_Paraphrase', 'BOSS_Sentiment_ID_T5_Paraphrase', 'BOSS_Sentiment_ID_T5_ICR', 'BOSS_Sentiment_ID_Falcon_Paraphrase', 'BOSS_Sentiment_ID_Falcon_ICR', 'BOSS_Toxicity_ID_BERT_Insert', 'BOSS_Toxicity_ID_BERT_Substitute', 'BOSS_Toxicity_ID_BERT_Translate', 'BOSS_Toxicity_ID_BERT_Paraphrase', 'BOSS_Toxicity_ID_T5_Insert', 'BOSS_Toxicity_ID_T5_Substitute', 'BOSS_Toxicity_ID_T5_Translate', 'BOSS_Toxicity_ID_T5_Paraphrase', 'BOSS_Toxicity_ID_T5_ICR', 'BOSS_Toxicity_ID_Falcon_Substitute', 'BOSS_Toxicity_ID_Falcon_Translate', 'BOSS_Toxicity_ID_Falcon_Paraphrase', 'BOSS_Toxicity_ID_Falcon_ICR', 'BOSS_Toxicity_ImplicitHate_Falcon_Paraphrase', 'BOSS_Toxicity_ImplicitHate_Falcon_ICR', 'AgNewsTweets_Tweets_Falcon_ICR']
{
    "BOSS_Sentiment_ID_BERT_Insert": "data/BOSS_Sentiment_ID_BERT_Insert/validation_aug_insert_random_16_Kyle1668/boss-sentiment-bert-base-uncased_style_logs.table.json",
 

In [6]:
column_renaming = {
    "entropy": "tta_prediction_entropy",
    "mean probs": "tta_mean_class_probs",
    "all probs": "tta_all_class_probs",
    "all entropies": "tta_all_class_entropy",
    "latency": "tta_inference_latency",
    "input": "augmentations",
    "original_input": "original_text",
    "judgment": "tta_predicted_class",
    "original judgment": "original_predicted_class",
    "original entropy": "original_prediction_entropy",
    "entropy decrease": "prediction_entropy_decrease",
    "entropy decreased": "prediction_entropy_decreased",
}
columns_to_drop = ["style prompt", "mean exemplar distance", "prompt"]
columns_order = ["outcome", "original_text", "augmentations", "original_predicted_class", "tta_predicted_class", "label", "tta_inference_latency", "original_prediction_entropy", "tta_prediction_entropy", "prediction_entropy_decreased", "prediction_entropy_decrease", "tta_mean_class_probs", "tta_all_class_probs", "tta_all_class_entropy", ]

In [7]:
def parse_augmentations(augmentations):
    if augmentations is None or len(augmentations) == 0:
        return []

    return [aug.replace("<aug>", "").replace("</aug>", "") for aug in augmentations[:4]]

formatted_augs = parse_augmentations([ "<aug>too not much of anything this well - well acted but dangerously slow thriller feels somehow like simply a terrible preamble to a bigger, much more too complicated horror story, one reality that never materializes.</aug>", "<aug>one too too much apprehension of this well - being acted but dangerously still slow thriller feels like only a preamble to a seemingly bigger, yet more more complicated story, and one that never materializes.</aug>", "<aug>too much of this a well - well acted but dangerously slow thriller film feels like a big preamble opening to a somehow bigger, more complicated dream story, the one concept that almost never materializes.</aug>", "<aug>too am much of this this well - acted but dangerously slow paced thriller feels somehow like a prime preamble to creating a bigger, more painfully complicated story, but one one that never ever materializes.</aug>", "<aug>Too much of this well-acted but dangerously slow thriller feels like a preamble to a bigger , more complicated story , one that never materializes .</aug>", "<aug>Too much of this well-acted but dangerously slow thriller feels like a preamble to a bigger , more complicated story , one that never materializes .</aug>" ])

print(formatted_augs)
print(len(formatted_augs))

['too not much of anything this well - well acted but dangerously slow thriller feels somehow like simply a terrible preamble to a bigger, much more too complicated horror story, one reality that never materializes.', 'one too too much apprehension of this well - being acted but dangerously still slow thriller feels like only a preamble to a seemingly bigger, yet more more complicated story, and one that never materializes.', 'too much of this a well - well acted but dangerously slow thriller film feels like a big preamble opening to a somehow bigger, more complicated dream story, the one concept that almost never materializes.', 'too am much of this this well - acted but dangerously slow paced thriller feels somehow like a prime preamble to creating a bigger, more painfully complicated story, but one one that never ever materializes.']
4


In [9]:
combined_datset = DatasetDict()
for split in tqdm(local_paths):
    # print(f"Processing {split}")
    path = local_paths[split]
    json_logs = json.load(open(path))
    frame = pd.DataFrame(data=json_logs["data"], columns=json_logs["columns"])
    frame = frame.rename(columns=column_renaming)
    frame = frame.drop(columns=columns_to_drop)
    frame["augmentations"] = frame["augmentations"].apply(parse_augmentations)

    if "BERT" not in split:
        for col in columns_order:
            if col not in frame.columns:
                if col == "original_prediction_entropy" or col == "tta_prediction_entropy" or col == "prediction_entropy_decrease":
                    frame[col] = float("nan")
                elif col == "prediction_entropy_decreased":
                    frame[col] = False
                elif col == "tta_all_class_probs":
                    frame[col] = frame.apply(lambda x: [[float(0.0)]], axis=1)
                elif "probs" in col:
                    frame[col] = frame.apply(lambda x: [float(0.0)], axis=1)
                elif col == "tta_all_class_entropy":
                    frame[col] = frame.apply(lambda x: [float(0.0)], axis=1)


    frame = frame[columns_order]
    combined_datset[split] = Dataset.from_pandas(frame)

combined_datset

  0%|          | 0/127 [00:00<?, ?it/s]

100%|██████████| 127/127 [02:30<00:00,  1.19s/it]


DatasetDict({
    BOSS_Sentiment_ID_BERT_Insert: Dataset({
        features: ['outcome', 'original_text', 'augmentations', 'original_predicted_class', 'tta_predicted_class', 'label', 'tta_inference_latency', 'original_prediction_entropy', 'tta_prediction_entropy', 'prediction_entropy_decreased', 'prediction_entropy_decrease', 'tta_mean_class_probs', 'tta_all_class_probs', 'tta_all_class_entropy'],
        num_rows: 38904
    })
    BOSS_Sentiment_ID_BERT_ICR: Dataset({
        features: ['outcome', 'original_text', 'augmentations', 'original_predicted_class', 'tta_predicted_class', 'label', 'tta_inference_latency', 'original_prediction_entropy', 'tta_prediction_entropy', 'prediction_entropy_decreased', 'prediction_entropy_decrease', 'tta_mean_class_probs', 'tta_all_class_probs', 'tta_all_class_entropy'],
        num_rows: 38904
    })
    BOSS_Sentiment_ID_T5_Insert: Dataset({
        features: ['outcome', 'original_text', 'augmentations', 'original_predicted_class', 'tta_predicted_cla

In [10]:
combined_datset.push_to_hub("LLM-TTA-Augmentation-Logs")

Creating parquet from Arrow format: 100%|██████████| 21/21 [00:00<00:00, 211.56ba/s]
